## Import

In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
from pathlib import Path
import time

In [3]:
from pysmFISH.pipeline import Pipeline

/Users/simone.codeluppi/opt/miniconda3/envs/pysmFISH-segmentation/lib/python3.8/site-packages/dask_jobqueue/core.py:19: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/Users/simone.codeluppi/opt/miniconda3/envs/pysmFISH-segmentation/lib/python3.8/site-packages/dask_jobqueue/core.py:19: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/Users/simone.codeluppi/opt/miniconda3/envs/pysmFISH-segmentation/lib/python3.8/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


## Prepare the folders for experimental tests

In [ ]:
!mkdir 'processing_folder/LBEXP20210718_EEL_Mouse_448_2'
!cp -r 'data_storage/LBEXP20210718_EEL_Mouse_448_2/fresh_tissue' 'processing_folder/LBEXP20210718_EEL_Mouse_448_2/'
!cp -r 'data_storage/required_folders_storage/codebooks' 'processing_folder/'
!cp -r 'data_storage/required_folders_storage/probes_sets' 'processing_folder/'
!cp -r 'data_storage/LBEXP20210718_EEL_Mouse_448_2/210906_21_07_09_LBEXP20210718_EEL_Mouse_448_2_img_data_dataset.parquet' 'processing_folder/LBEXP20210718_EEL_Mouse_448_2/'

## Initialize the pipeline

In [4]:
# Enter the required parameters
experiment_fpath = Path('/Users/simone.codeluppi/Documents/data_analysis_jlabs_sc/notebooks_pysmFISH/processing_folder/LBEXP20210718_EEL_Mouse_448_2')
dataset_name = '210906_21_07_09_LBEXP20210718_EEL_Mouse_448_2_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'local'

In [6]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        cores=3,
        memory='12GB',
        chunk_size = 6,
        dataset_path = experiment_fpath / dataset_name)

CPU times: user 124 µs, sys: 6 µs, total: 130 µs
Wall time: 134 µs


## Run the required steps

In [7]:
%%time
running_pipeline.run_setup()
running_pipeline.run_cluster_activation()
running_pipeline.run_parsing()
running_pipeline.run_required_steps()

distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist
distributed.worker - INFO - notebooks already exist


CPU times: user 2.51 s, sys: 479 ms, total: 2.99 s
Wall time: 7.3 s


In [ ]:
running_pipeline.client

## Process the images of the fresh tissue

In [ ]:
%%time
running_pipeline.processing_fresh_tissue_step(parsing=False)

In [ ]:
running_pipeline.client.close()

In [ ]:
running_pipeline.cluster.close()

## Remove overlapping objects

In [3]:
from pysmFISH.fov_processing import segmentation_NN_fov
from pysmFISH.io import load_general_zarr
import zarr
from pysmFISH.utils import convert_from_uint16_to_float64
import pickle
from pathlib import Path
import pandas as pd
from pysmFISH.stitching import organize_square_tiles
from skimage import measure
import numpy as np
from pysmFISH.data_models import Dataset
from dask.distributed import Client, LocalCluster
from pysmFISH.processing_cluster_setup import local_cluster_setup
from pysmFISH.segmentation import remove_overlapping_obj

/Users/simone.codeluppi/opt/miniconda3/envs/pysmFISH-segmentation/lib/python3.8/site-packages/dask_jobqueue/core.py:19: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/Users/simone.codeluppi/opt/miniconda3/envs/pysmFISH-segmentation/lib/python3.8/site-packages/dask_jobqueue/core.py:19: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/Users/simone.codeluppi/opt/miniconda3/envs/pysmFISH-segmentation/lib/python3.8/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


In [ ]:
cores = 4
memory = 6
cluster = local_cluster_setup(cores=cores,memory=memory)
client = Client(cluster)

In [15]:
client = Client()

In [91]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 64.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60476,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: 1 minute ago,Total memory: 64.00 GiB
Comm: tcp://127.0.0.1:60658,Total threads: 4
Dashboard: http://127.0.0.1:60665/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:60478,


In [ ]:
- Bring in the information from the experiment.dataset to fill the blanks and the nuclei
- Add the creation of the segmentation folder to the path
- adjust to the segmentation_output from the experiment-path in the registered coords obj
- maybe add in the pipeline attributes the size of the overlapping objects to remove
- check with Lars for segmentation data

In [83]:
def load_segmented_data(fov_subdataset,experiment_path):
    experiment_path = Path(experiment_path)
    experiment_name = fov_subdataset.experiment_name
    segmented_file_path = experiment_path / 'fresh_tissue' / 'fresh_tissue_segmented_nuclei_data.zarr'
    st = zarr.DirectoryStore(segmented_file_path)
    root = zarr.group(store=st,overwrite=False)
    grp_name = (
            experiment_name
            + "_segmetented_fresh_tissue_fov_"
            + str(fov_subdataset.fov_num)
        )
        
    dataset_name = "segmentation_mask_fov_" + str(fov_subdataset.fov_num)
    
    mask = root[grp_name][dataset_name][...]
    return mask

In [84]:
def load_stitched_segmented_data(fov_subdataset,segmentation_output_path):
    segmented_region_img = pickle.load(open(segmentation_output_path / ('registered_objs_dict_fov_' + str(fov_subdataset.fov_num) + '.pkl'), 'rb'))

    obj_segmented_img = {el:coords_dict['stitched_coords'] for (el,coords_dict) in segmented_region_img.items()}
    return obj_segmented_img

def load_stitched_segmented_data_full_dict(fov_subdataset,segmentation_output_path):
    segmented_region_dict = pickle.load(open(segmentation_output_path / ('registered_objs_dict_fov_' + str(fov_subdataset.fov_num) + '.pkl'), 'rb'))
    return segmented_region_dict
    

In [85]:
def stitch_using_coords_general_segmented_objects(fov,obj_dict,tile_corners_coords_pxl,reference_corner_fov_position,metadata):
    """
    Tiles are placed directly on the position indicated by the microscope
    coords
    """

    r_coords = tile_corners_coords_pxl[fov,0]
    c_coords = tile_corners_coords_pxl[fov,1]
    
    if obj_dict:
        
        if reference_corner_fov_position == 'top-left':
            for el,coords_dict in obj_dict.items():
                coords_dict['stitched_coords'] = np.vstack([r_coords + coords_dict['original_coords'][:,0],
                                                            c_coords + coords_dict['original_coords'][:,1]]).T
                coords_dict['stitched_centroid'] = np.vstack([r_coords + coords_dict['original_centroid'][0],
                                                            c_coords + coords_dict['original_centroid'][1]]).T

        elif reference_corner_fov_position == 'top-right':
            for el,coords_dict in obj_dict.items():
                coords_dict['stitched_coords'] = np.vstack([r_coords + coords_dict['original_coords'][:,0],
                                                            c_coords - (metadata['img_width'] -coords_dict['original_coords'][:,1])]).T
                coords_dict['stitched_centroid'] = np.vstack([r_coords + coords_dict['original_centroid'][0],
                                                            c_coords - (metadata['img_width'] -coords_dict['original_centroid'][1])]).T

        elif reference_corner_fov_position == 'bottom_left':
            for el,coords_dict in obj_dict.items():
                coords_dict['stitched_coords'] = np.vstack([r_coords + (metadata['img_height'] -coords_dict['original_coords'][:,0]),
                                                            c_coords + coords_dict['original_coords'][:,1]]).T
                coords_dict['stitched_centroid'] = np.vstack([r_coords + (metadata['img_height'] -coords_dict['original_centroid'][0]),
                                                            c_coords + coords_dict['original_centroid'][1]]).T

    return obj_dict

In [86]:
def register_coords_obj(fov_subdataset, experiment_fpath, segmentation_output_path,
                        tile_corners_coords_global_pxl,metadata,reference_corner_fov_position='top-right'):
    
    mask = load_segmented_data(fov_subdataset,experiment_fpath)
    segmentation_output_path = Path(experiment_fpath) / 'fresh_tissue' / 'segmentation'
    
    segmented_regions = measure.regionprops(mask)
    segmented_regions_dict = {}
    for prop in segmented_regions:
        segmented_regions_dict[str(fov_subdataset.fov_num)+'-'+str(prop.label)] = {}
        segmented_regions_dict[str(fov_subdataset.fov_num)+'-'+str(prop.label)]['original_coords']=prop.coords
        segmented_regions_dict[str(fov_subdataset.fov_num)+'-'+str(prop.label)]['stitched_coords']= np.nan
        segmented_regions_dict[str(fov_subdataset.fov_num)+'-'+str(prop.label)]['original_centroid']= prop.centroid
        segmented_regions_dict[str(fov_subdataset.fov_num)+'-'+str(prop.label)]['stitched_centroid']= prop.centroid
        segmented_regions_dict[str(fov_subdataset.fov_num)+'-'+str(prop.label)]['area']= prop.area
        segmented_regions_dict = stitch_using_coords_general_segmented_objects(fov_subdataset.fov_num,segmented_regions_dict,
                                                                         tile_corners_coords_global_pxl,reference_corner_fov_position,metadata)
    pickle.dump(segmented_regions_dict,open(segmentation_output_path / ('registered_objs_dict_fov_' + str(fov_subdataset.fov_num) + '.pkl'), 'wb'))
#    return segmented_regions_dict

In [48]:
dataset_name = '220415_16_38_11_20210718_124116_843__ChannelCy3_Nuclei_Seq0001_img_data_dataset.parquet'
dataset_nuclei = Dataset()
dataset_nuclei.load_dataset(Path(experiment_fpath) / 'fresh_tissue' / dataset_name)
dataset_nuclei.dataset['processing_type'] = 'undefined'
dataset_nuclei.dataset['overlapping_percentage'] = 5
dataset_nuclei.dataset['machine'] = 'ROBOFISH2'

metadata_nuclei = dataset_nuclei.collect_metadata(dataset_nuclei.dataset)
fov_subdataset = dataset_nuclei.dataset.iloc[0]
tile_corners_coords_global_pxl = pickle.load(open(Path(experiment_fpath) / 'fresh_tissue' / 'results' / 'global_stitched_coords.pkl','rb'))
nuclei_org_tiles = organize_square_tiles(experiment_fpath,dataset_nuclei.dataset,metadata_nuclei,round_num=1)
nuclei_org_tiles.run_tiles_organization()
a = nuclei_org_tiles.tile_corners_coords_pxl
nuclei_org_tiles.tile_corners_coords_pxl = tile_corners_coords_global_pxl
nuclei_org_tiles.determine_overlapping_regions()
reference_corner_fov_position = nuclei_org_tiles.reference_corner_fov_position

In [77]:
mc = np.load('/Users/simone.codeluppi/Downloads/microscope_tile_corners_coords_pxl.npy')
gb = pickle.load(open('/Users/simone.codeluppi/Downloads/global_stitched_coords.pkl','rb'))
gs = pickle.load(open('/Users/simone.codeluppi/Downloads/stitching_global_shift.pkl','rb'))

In [79]:
mc.shape

(305, 2)

In [72]:
mc[0,:]

array([27235.63144851, 35026.91631974])

In [73]:
gb[0,:]

array([39568.71043075, 39576.43977281])

In [80]:
gs[0:20,:]

array([[  0.,   0.],
       [  3.,   9.],
       [  2.,  18.],
       [  2.,  27.],
       [  2.,  37.],
       [  0.,  46.],
       [  0.,  55.],
       [ -1.,  64.],
       [ -2.,  73.],
       [ -3.,  82.],
       [ -9.,  93.],
       [ -9.,  82.],
       [-10.,  72.],
       [-10.,  63.],
       [ -8.,  54.],
       [ -7.,  45.],
       [ -6.,  37.],
       [ -6.,  26.],
       [ -5.,  18.],
       [ -4.,   8.]])

In [51]:
nuclei_org_tiles.tile_corners_coords_pxl[0,:]

array([27235.63144851, 35026.91631974])

In [53]:
tile_corners_coords_global_pxl[0,:]

array([27235.63144851, 35026.91631974])

In [ ]:
all_futures = []
    for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
        future = client.submit(register_coords_obj,fov_subdataset,
                               experiment_fpath,
                               segmentation_output_path,
                              tile_corners_coords_global_pxl,
                              metadata_nuclei,
                               reference_corner_fov_position='top-right')
        all_futures.append(future)
    _ = client.gather(all_futures)
    

In [17]:
from pysmFISH.segmentation import register_coords_obj

In [ ]:

    all_futures = []
    for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
        future = client.submit(register_coords_obj,fov_subdataset,
                               experiment_fpath,
                               segmentation_output_path,
                              tile_corners_coords_global_pxl,
                              metadata_nuclei,
                               reference_corner_fov_position='top-right')
        all_futures.append(future)
    _ = client.gather(all_futures)
    
    

In [25]:
def create_label_image(experiment_fpath,
                       segmentation_output_path,
                       ds_nuclei,
                       nuclei_org_tiles,
                       nuclei_adjusted_coords,
                       client,
                       min_overlapping_pixels_segmentation = 20):
    
    # Get objects properties and register to the data
    nuclei_org_tiles.tile_corners_coords_pxl = tile_corners_coords_global_pxl
    reference_corner_fov_position = nuclei_org_tiles.reference_corner_fov_position
        
    
    
    # Remove overlapping objects paralle
    all_futures = []
    for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
        future = client.submit(remove_overlapping_obj,
                               fov_subdataset,
                               nuclei_org_tiles,
                              segmentation_output_path,
                              min_overlapping_pixels_segmentation)
        all_futures.append(future)
    all_outputs = client.gather(all_futures)
    
    
    # tmp_saving
    pickle.dump(all_outputs,open(segmentation_output_path / ('output_removal_tmp.pkl'), 'wb'))
    
    all_obj_original = {}
    for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
        segmented_region_img = load_stitched_segmented_data_full_dict(fov_subdataset,segmentation_output_path)
        all_obj_original.update(segmented_region_img)

    all_remove_objs_list = []
    all_add_objs = {}
    for rem, obj in all_outputs:
        all_remove_objs_list.append(rem)
        all_add_objs.update(obj)
    all_remove_objs_list = [val for el in all_remove_objs_list for val in el ]

    for cell_id, coords in all_add_objs.items():
        all_obj_original[cell_id]['stitched_coords'] = coords

    all_remove_objs_list_tmp=all_remove_objs_list.copy()
    for obj in all_obj_original:
        if obj in all_remove_objs_list_tmp:
            all_remove_objs_list_tmp.remove(obj)

    for obj in all_remove_objs_list_tmp:
        all_obj_original.pop(obj,None)
        
    
    pickle.dump(all_obj_original,open(segmentation_output_path / ('segmented_objects_dict.pkl'), 'wb'))
    
    # Create image
    # Calculate image size 

    row_min = tile_corners_coords_global_pxl[:,0].min()
    col_min = tile_corners_coords_global_pxl[:,1].min()
    row_max = tile_corners_coords_global_pxl[:,0].max()
    col_max = tile_corners_coords_global_pxl[:,1].max()

    nrows = np.int(row_max - row_min + metadata_nuclei['img_height'])
    ncols = np.int(col_max - col_min + metadata_nuclei['img_width'])

    img = np.zeros([nrows,ncols])

    for idx, data_dict in enumerate(all_obj_original.items()):
        coords = data_dict[1]['stitched_coords']
        coords = coords -[row_min,col_min]
        coords = coords.astype(int)
        img[coords[:,0],coords[:,1]] = idx

    zarr_fpath = segmentation_output_path / 'image_segmented_labels.zarr'
    store = zarr.DirectoryStore(zarr_fpath,'w')
    grp = zarr.group(store=store,overwrite=True)
    grp.create_dataset(name='segmented_labels_image', data=img)

In [11]:
experiment_fpath = Path('/Users/simone.codeluppi/Documents/data_analysis_jlabs_sc/notebooks_pysmFISH/processing_folder/LBEXP20210718_EEL_Mouse_448_2')
segmentation_output_path = experiment_fpath / 'fresh_tissue' / 'segmentation'

In [21]:
from pysmFISH.segmentation import load_stitched_segmented_data_full_dict

In [27]:
%time
create_label_image(experiment_fpath,
                       segmentation_output_path,
                       dataset_nuclei,
                       nuclei_org_tiles,
                       tile_corners_coords_global_pxl,
                       client)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 17.9 µs


In [28]:
def create_low_mag_beads_target(experiment_fpath):
    all_counts_collected = []
    counts_paths = (Path(experiment_fpath) / 'fresh_tissue' /'results').glob('*counts_beads_fresh_tissue_decoded_fov*')
    for count_fpath in counts_paths:
        data = pd.read_parquet(count_fpath)
        data = data.loc[:, ['r_px_global_stitched_nuclei','c_px_global_stitched_nuclei']]
        all_counts_collected.append(data)
    all_counts_collected = pd.concat(all_counts_collected,axis=0)
    target = all_counts_collected.loc[:,['r_px_global_stitched_nuclei','c_px_global_stitched_nuclei']].to_numpy()
    return target

In [29]:
beads_df = create_low_mag_beads_target(experiment_fpath)


In [30]:
np.save(segmentation_output_path /'beads-40X.npy',beads_df)

In [111]:
def create_high_mag_beads_RNA_source(experiment_path,hamming_distance=3):
    all_beads = []
    all_RNA = []
    filter_distance = hamming_distance / 16
    counts_paths = (Path(experiment_path) / 'results').glob('*_decoded_fov_*')
    for count_fpath in counts_paths:
        data = pd.read_parquet(count_fpath)
        data_beads = data.loc[(data.channel == 'Europium') &
                        (data.mapped_beads_type == 'large') &
                        (data.round_num == 1), ['r_px_global_stitched','c_px_global_stitched']]
        
        data_RNA = data.loc[(data.channel != 'Europium') &
                            (data.hamming_distance < filter_distance), ['decoded_genes','r_px_global_stitched','c_px_global_stitched']]
        all_beads.append(data_beads)
        all_RNA.append(data_RNA)
    all_beads = pd.concat(all_beads,axis=0)
    all_RNA = pd.concat(all_RNA,axis=0)
    source_beads = all_beads.loc[:,['r_px_global_stitched','c_px_global_stitched']].to_numpy()
    source_RNA_df = all_RNA.loc[:,['decoded_genes','r_px_global_stitched','c_px_global_stitched']].to_numpy()
    return source_beads, source_RNA_df

In [112]:
source_beads, source_RNA_df = create_high_mag_beads_RNA_source(experiment_fpath,hamming_distance=3)

(12562991, 3)

In [115]:
np.save(Path(experiment_fpath) / 'results' /'beads-60X-global.npy',beads_df)

In [116]:
def create_high_mag_beads_RNA_source_micro(experiment_path,hamming_distance=3):
    all_beads = []
    all_RNA = []
    filter_distance = hamming_distance / 16
    counts_paths = (Path(experiment_path) / 'results').glob('*_decoded_fov_*')
    for count_fpath in counts_paths:
        data = pd.read_parquet(count_fpath)
        data_beads = data.loc[(data.channel == 'Europium') &
                        (data.mapped_beads_type == 'large') &
                        (data.round_num == 1), ['r_px_microscope_stitched','c_px_microscope_stitched']]
        
        data_RNA = data.loc[(data.channel != 'Europium') &
                            (data.hamming_distance < filter_distance), ['decoded_genes','r_px_microscope_stitched','c_px_microscope_stitched']]
        all_beads.append(data_beads)
        all_RNA.append(data_RNA)
    all_beads = pd.concat(all_beads,axis=0)
    all_RNA = pd.concat(all_RNA,axis=0)
    source_beads = all_beads.loc[:,['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()
    source_RNA_df = all_RNA.loc[:,['decoded_genes','r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()
    return source_beads, source_RNA_df

In [117]:
source_beads, source_RNA_df = create_high_mag_beads_RNA_source_micro(experiment_fpath,hamming_distance=3)

In [118]:
source_beads.shape

(14806, 2)

In [119]:
np.save(Path(experiment_fpath) / 'results' /'beads-60X-microscope.npy',beads_df)

In [109]:
df = pd.read_parquet(Path(experiment_fpath) / 'results' / 'LBEXP20210718_EEL_Mouse_448_2_decoded_fov_554.parquet')

In [105]:
df.loc[(df.channel == 'Europium')&
       (df.round_num == 1) &
       (df.mapped_beads_type,:]

,r_px_original,c_px_original,dot_id,dot_intensity,selected_thr,barcode,barcode_length,channel,codebook,experiment_name,...,bit_9_intensity,bit_10_intensity,bit_11_intensity,bit_12_intensity,bit_13_intensity,bit_14_intensity,bit_15_intensity,bit_16_intensity,r_px_global_stitched,c_px_global_stitched


In [100]:
df.mapped_beads_type.unique()

array(['large', 'small', None], dtype=object)

In [110]:
df.columns

Index(['r_px_original', 'c_px_original', 'dot_id', 'dot_intensity',
       'selected_thr', 'barcode', 'barcode_length', 'channel', 'codebook',
       'experiment_name', 'experiment_type', 'fov_acquisition_coords_x',
       'fov_acquisition_coords_y', 'fov_acquisition_coords_z', 'fov_name',
       'fov_num', 'grp_name', 'img_height', 'img_width', 'machine', 'operator',
       'overlapping_percentage', 'pipeline', 'pixel_microns',
       'probe_fasta_name', 'processing_type', 'round_num', 'species',
       'start_date', 'stitching_channel', 'stitching_type', 'strain',
       'target_name', 'tissue', 'total_fovs', 'zstack', 'raw_data_location',
       'mapped_beads_type', 'r_px_registered', 'c_px_registered', 'r_shift_px',
       'c_shift_px', 'min_number_matching_dots_registration',
       'r_px_microscope_stitched', 'c_px_microscope_stitched',
       'barcode_reference_dot_id', 'raw_barcodes', 'decoded_genes',
       'hamming_distance', 'number_positive_bits',
       'barcodes_extractio

In [6]:
all_counts_collected = []
counts_paths = (Path('/Users/simone.codeluppi/Downloads/') /'results').glob('*counts_beads_fresh_tissue_decoded_fov*')
for count_fpath in counts_paths:
    data = pd.read_parquet(count_fpath)
    data = data.loc[:, ['r_px_original','c_px_original']]
    all_counts_collected.append(data)
all_counts_collected = pd.concat(all_counts_collected,axis=0)
target = all_counts_collected.loc[:,['r_px_original','c_px_original']].to_numpy()

In [7]:
target.shape

(9301, 2)

In [47]:
seg = pickle.load(open('/Users/simone.codeluppi/Downloads/segmented_objects_dict.pkl','rb' ))

In [11]:
# Serial
for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
    register_coords_obj(fov_subdataset,experiment_fpath,tile_corners_coords_global_pxl,metadata_nuclei,reference_corner_fov_position='top-right')

In [ ]:
# Parallel
all_futures = []
for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
    future = client.submit(register_coords_obj,fov_subdataset,
                           experiment_fpath,
                           segmentation_output_path,
                          tile_corners_coords_global_pxl,
                          metadata_nuclei,
                           reference_corner_fov_position='top-right')
    all_futures.append(future)
_ = client.gather(all_futures)

In [28]:
# Remove overlapping objects serial
segmentation_output_path = Path(experiment_fpath) / 'fresh_tissue' / 'segmentation'
min_overlapping_pixels_segmentation = 20
all_output = []
for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
    all_output.append(remove_overlapping_obj(
                           fov_subdataset,
                           nuclei_org_tiles,
                          segmentation_output_path,
                          min_overlapping_pixels_segmentation))


In [ ]:
# Remove overlapping objects paralle
min_overlapping_pixels_segmentation = 20
all_futures = []
for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
    future = client.submit(remove_overlapping_obj,
                           fov_subdataset,
                           nuclei_org_tiles,
                          segmentation_output_path,
                          min_overlapping_pixels_segmentation)
    all_futures.append(future)
all_outputs = client.gather(all_futures)


In [30]:
# tmp_saving
pickle.dump(all_output,open(segmentation_output_path / ('output_removal_tmp.pkl'), 'wb'))

In [152]:
all_obj_original = {}
for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
    segmented_region_img = load_stitched_segmented_data_full_dict(fov_subdataset,segmentation_output_path)
    all_obj_original.update(segmented_region_img)

In [153]:
all_obj_original = {}
for idx,fov_subdataset in dataset_nuclei.dataset.iterrows():
    segmented_region_img = load_stitched_segmented_data_full_dict(fov_subdataset,segmentation_output_path)
    all_obj_original.update(segmented_region_img)

all_remove_objs_list = []
all_add_objs = {}
for rem, obj in all_output:
    all_remove_objs_list.append(rem)
    all_add_objs.update(obj)
all_remove_objs_list = [val for el in all_remove_objs_list for val in el ]

for cell_id, coords in all_add_objs.items():
    all_obj_original[cell_id]['stitched_coords'] = coords

all_remove_objs_list_tmp=all_remove_objs_list.copy()
for obj in all_obj_original:
    if obj in all_remove_objs_list_tmp:
        all_remove_objs_list_tmp.remove(obj)

for obj in all_remove_objs_list_tmp:
    all_obj_original.pop(obj,None)
        

In [125]:
for obj in all_remove_objs_list:
    all_obj_original.pop(obj,None)

In [142]:
rem

[]

In [51]:
pickle.dump(all_obj_original,open(segmentation_output_path / ('segmented_objects_dict.pkl'), 'wb'))

In [156]:
# Create image
# Calculate image size 

row_min = tile_corners_coords_global_pxl[:,0].min()
col_min = tile_corners_coords_global_pxl[:,1].min()
row_max = tile_corners_coords_global_pxl[:,0].max()
col_max = tile_corners_coords_global_pxl[:,1].max()

nrows = np.int(row_max - row_min + metadata_nuclei['img_height'])
ncols = np.int(col_max - col_min + metadata_nuclei['img_width'])

img = np.zeros([nrows,ncols])

for idx, data_dict in enumerate(all_obj_original.items()):
    coords = data_dict[1]['stitched_coords']
    coords = coords -[row_min,col_min]
    coords = coords.astype(int)
    img[coords[:,0],coords[:,1]] = idx
    
zarr_fpath = segmentation_output_path / 'image_segmented_labels.zarr'
store = zarr.DirectoryStore(zarr_fpath,'w')
grp = zarr.group(store=store,overwrite=True)
grp.create_dataset(name='segmented_labels_image', data=img)

<zarr.core.Array '/segmented_labels_image' (29286, 60415) float64>

In [157]:
img.max()

231812.0

In [114]:
row_min = tile_corners_coords_global_pxl[:,0].min()
col_min = tile_corners_coords_global_pxl[:,1].min()
row_max = tile_corners_coords_global_pxl[:,0].max()
col_max = tile_corners_coords_global_pxl[:,1].max()

nrows = np.int(row_max - row_min + metadata_nuclei['img_height'])
ncols = np.int(col_max - col_min + metadata_nuclei['img_width'])

img = np.zeros([nrows,ncols])
for idx, data_dict in enumerate(all_obj_original.items()):
    coords = data_dict[1]['stitched_coords']
    coords = coords -[row_min,col_min]
    coords = coords.astype(int)
    img[coords[:,0],coords[:,1]] = idx

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [117]:
all_add_objs['0-5']

array([[28765, 33007],
       [28765, 33008],
       [28765, 33009],
       ...,
       [28794, 33014],
       [28794, 33015],
       [28795, 33012]])

In [79]:
zarr_fpath = segmentation_output_path / 'image_segmented_labels.zarr'
store = zarr.DirectoryStore(zarr_fpath,'w')
grp = zarr.group(store=store,overwrite=True)
grp.create_dataset(name='segmented_labels_image', data=img)

<zarr.core.Array '/segmented_labels_image' (29286, 60415) float64>

In [111]:
all_add_objs

{'0-5': array([[28765, 33007],
        [28765, 33008],
        [28765, 33009],
        ...,
        [28794, 33014],
        [28794, 33015],
        [28795, 33012]]),
 '1-39': array([[27516, 31036],
        [27516, 31037],
        [27516, 31038],
        ...,
        [27549, 31041],
        [27550, 31036],
        [27550, 31037]]),
 '1-44': array([[28074, 31076],
        [28074, 31077],
        [28074, 31078],
        [28074, 31079],
        [28074, 31080],
        [28074, 31081],
        [28074, 31082],
        [28074, 31083],
        [28075, 31074],
        [28075, 31075],
        [28075, 31076],
        [28075, 31077],
        [28075, 31078],
        [28075, 31079],
        [28075, 31080],
        [28075, 31081],
        [28075, 31082],
        [28075, 31083],
        [28075, 31084],
        [28076, 31073],
        [28076, 31074],
        [28076, 31075],
        [28076, 31076],
        [28076, 31077],
        [28076, 31078],
        [28076, 31079],
        [28076, 31080],
        [28

In [ ]:
from pysmFISH.segmentation_NN import Segmenation_NN

In [ ]:
segmentation = Segmenation_NN('stardist')

In [ ]:
mask = segmentation.segment(img)

In [ ]:
from stardist.models import StarDist2D 
from csbdeep.utils import normalize

In [ ]:
#Instantiate model    
model = StarDist2D.from_pretrained('2D_versatile_fluo')
        
#Segment
mask, _ = model.predict_instances(normalize(img),verbose=True)

In [ ]:
model = StarDist2D.from_pretrained('2D_versatile_fluo')

In [ ]:
model.predict_instances

In [ ]:
from cellpose import models

In [10]:
running_pipeline.tiles_org.reference_corner_fov_position

'top-right'

In [12]:
running_pipeline.metadata

{'list_all_fovs': array([  0,   1,  10, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
         11, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,  12, 120,
        121, 122, 123, 124, 125, 126, 127, 128, 129,  13, 130, 131, 132,
        133, 134, 135, 136, 137, 138, 139,  14, 140, 141, 142, 143, 144,
        145, 146, 147, 148, 149,  15, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159,  16, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169,  17, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179,  18,
        180, 181, 182, 183, 184, 185, 186, 187, 188, 189,  19, 190, 191,
        192, 193, 194, 195, 196, 197, 198, 199,   2,  20, 200, 201, 202,
        203, 204, 205, 206, 207, 208, 209,  21, 210, 211, 212, 213, 214,
        215, 216, 217, 218, 219,  22, 220, 221, 222, 223, 224, 225, 226,
        227, 228, 229,  23, 230, 231, 232, 233, 234, 235, 236, 237, 238,
        239,  24, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249,  25,
        250, 251, 252, 253, 254, 2

In [ ]:
running_pipeline.c